# Medic Attends and Completes Medical School

## As a result they are issued with a Primary Medical Qualification

Note for the time being we are modelling the medic as an ACA-Py agent although a more realistic scenario would use a mobile wallet.

![Zoom Cred](../zoom-cred-deps.png)

### Imports

In [3]:
from aries_cloudcontroller import AriesAgentController
import os
from termcolor import colored

### Initialise the Agent Controller

In [4]:
api_key = os.getenv("ACAPY_ADMIN_API_KEY")
admin_url = os.getenv("ADMIN_URL")

print(f"Initialising a controller with admin api at {admin_url} and an api key of {api_key}")
agent_controller = AriesAgentController(admin_url,api_key)

Initialising a controller with admin api at http://healthcare-professional-agent:3021 and an api key of MyMedicalAPIKey


### Start a Webhook Server

In [5]:
webhook_port = int(os.getenv("WEBHOOK_PORT"))
webhook_host = "0.0.0.0"

await agent_controller.init_webhook_server(webhook_host, webhook_port)

print(f"Listening for webhooks from agent at http://{webhook_host}:{webhook_port}")

Listening for webhooks from agent at http://0.0.0.0:3010


## Register Agent Event Listeners

You can see some examples within the webhook_listeners recipe. Copy any relevant cells across and customise as needed.

In [6]:
listeners = []

In [7]:
## YOUR LISTENERS HERE
def holder_handler(payload):
    connection_id = payload['connection_id']
    exchange_id = payload['credential_exchange_id']
    state = payload['state']
    role = payload['role']
    print("\n---------------------------------------------------\n")
    print("Handle Issue Credential Webhook")
    print(f"Connection ID : {connection_id}")
    print(f"Credential exchange ID : {exchange_id}")
    print("Agent Protocol Role : ", role)
    print("Protocol State : ", state )
    print("\n---------------------------------------------------\n")
    print("Handle Credential Webhook Payload")
    
    if state == "offer_received":
        print("Credential Offer Recieved")
        proposal = payload["credential_proposal_dict"]
        print("The proposal dictionary is likely how you would understand and display a credential offer in your application")
        print("\n", proposal)
        print("\n This includes the set of attributes you are being offered")
        attributes = proposal['credential_proposal']['attributes']
        print(attributes)
        ## YOUR LOGIC HERE
    elif state == "request_sent":
        print("\nA credential request object contains the commitment to the agents master secret using the nonce from the offer")
        ## YOUR LOGIC HERE
    elif state == "credential_received":
        print("Received Credential")
        ## YOUR LOGIC HERE
    elif state == "credential_acked":
        ## YOUR LOGIC HERE
        print("Credential Stored\n")
        credential = payload["credential"]
        print("\nThe referent acts as the identifier for retrieving the raw credential from the wallet")
        # Note: You would probably save this in your application database
        credential_referent = credential["referent"]
        print("Referent", credential_referent)
    


    
holder_listener = {
    "topic": "issue_credential",
    "handler": holder_handler
}

listeners.append(holder_listener)

In [8]:
# Receive connection messages
def connections_handler(payload):
    state = payload['state']
    connection_id = payload["connection_id"]
    their_role = payload["their_role"]
    routing_state = payload["routing_state"]
    
    print("----------------------------------------------------------")
    print("Connection Webhook Event Received")
    print("Connection ID : ", connection_id)
    print("State : ", state)
    print("Routing State : ", routing_state)
    print("Their Role : ", their_role)
    print("----------------------------------------------------------")

    if state == "invitation":
        # Your business logic
        print("invitation")
    elif state == "request":
        # Your business logic
        print("request")

    elif state == "response":
        # Your business logic
        print("response")
    elif state == "active":
        # Your business logic
        print(colored("Connection ID: {0} is now active.".format(connection_id), "green", attrs=["bold"]))
        

connection_listener = {
    "handler": connections_handler,
    "topic": "connections"
}

listeners.append(connection_listener)

In [9]:
agent_controller.register_listeners(listeners)

## Establish a Connection

Must establish connection with issuer before being able to receive credential. Holder modeled as invitee in this case. See recipes/connection.

In [10]:
invitation = {'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation', '@id': '776cd529-bab7-4450-8fdf-4da42ef8ac10', 'recipientKeys': ['6HYLGBRME7PnSpxwVP8Y32DCmNQHjFYpYupTisJEidrh'], 'label': 'Edinburgh Medical School', 'serviceEndpoint': 'https://ced6556c67a4.ngrok.io'}


In [11]:
auto_accept=False
alias=None

invite_response = await agent_controller.connections.receive_invitation(invitation, alias, auto_accept)
connection_id = invite_response["connection_id"]

----------------------------------------------------------
Connection Webhook Event Received
Connection ID :  8a8a6a54-eddf-4568-a5d1-a8359d2f9ca7
State :  invitation
Routing State :  none
Their Role :  inviter
----------------------------------------------------------
invitation
----------------------------------------------------------
Connection Webhook Event Received
Connection ID :  8a8a6a54-eddf-4568-a5d1-a8359d2f9ca7
State :  request
Routing State :  none
Their Role :  inviter
----------------------------------------------------------
request
----------------------------------------------------------
Connection Webhook Event Received
Connection ID :  8a8a6a54-eddf-4568-a5d1-a8359d2f9ca7
State :  response
Routing State :  none
Their Role :  inviter
----------------------------------------------------------
response
----------------------------------------------------------
Connection Webhook Event Received
Connection ID :  8a8a6a54-eddf-4568-a5d1-a8359d2f9ca7
State :  active
Rout

In [ ]:
## TODO: Verify is Trusted Medical School?

## Medical Student Request Credential From Offer

The medical student can decide whether the attributes being offered are accurate, before requesting the final signature on a PMQ credential containing theseattributes.


In [12]:
state="offer_received"
records_response = await agent_controller.issuer.get_records(connection_id=connection_id, state=state)
record = records_response["results"][0]
record_id = record["credential_exchange_id"]


## Medic Verifies Details in Credential Offer and Requests When Satsified

In [13]:
await agent_controller.issuer.send_request_for_record(record_id)


---------------------------------------------------

Handle Issue Credential Webhook
Connection ID : 8a8a6a54-eddf-4568-a5d1-a8359d2f9ca7
Credential exchange ID : c300ad4c-76cc-4d46-980d-d240a12aeccd
Agent Protocol Role :  holder
Protocol State :  request_sent

---------------------------------------------------

Handle Credential Webhook Payload

A credential request object contains the commitment to the agents master secret using the nonce from the offer


{'credential_request_metadata': {'master_secret_blinding_data': {'v_prime': '19905399353666993540705342869019292995794374348159433990924186185313176093355060356542949769941471277489130260011364085405617830504829349851341487046085472804380495533491590444106127373411327210744182442207013926082334431460951187535159938730575028580857572147451374606752654030973460441883447734390903283900862626154161989263280783446536862807410855059905888363462414839976269823399027323787426233069311623393079254928730528500134514349771127925623221284781212559227446484568647941619445798566304419574260111856011022634967635412520679379066752691390663888071303360146227793987968471929708360001511914035466414126889863754748623515906208240',
   'vr_prime': None},
  'nonce': '512130146359083366558673',
  'master_secret_name': 'Medic'},
 'created_at': '2021-05-28 09:37:17.149499Z',
 'schema_id': '3jnXQcj9VLFjcUbtDVZZzV:2:Primary Medical Qualification:0.0.2',
 'credential_request': {'prover_did': 'EJAAQqjXQ7ggffRHqoKYh


---------------------------------------------------

Handle Issue Credential Webhook
Connection ID : 8a8a6a54-eddf-4568-a5d1-a8359d2f9ca7
Credential exchange ID : c300ad4c-76cc-4d46-980d-d240a12aeccd
Agent Protocol Role :  holder
Protocol State :  credential_received

---------------------------------------------------

Handle Credential Webhook Payload
Received Credential


## Medic Stores Received PMQ for later use


In [14]:
# Optionally specify an identifier to uniquely identify this credential within your agents wallet.
# You would likely want to save this somewhere.
# If not set a random one will be generated for you
credential_id = "PMQ"

store_cred_response = await agent_controller.issuer.store_credential(record_id, credential_id)


---------------------------------------------------

Handle Issue Credential Webhook
Connection ID : 8a8a6a54-eddf-4568-a5d1-a8359d2f9ca7
Credential exchange ID : c300ad4c-76cc-4d46-980d-d240a12aeccd
Agent Protocol Role :  holder
Protocol State :  credential_acked

---------------------------------------------------

Handle Credential Webhook Payload
Credential Stored


The referent acts as the identifier for retrieving the raw credential from the wallet
Referent PMQ


## Terminate Controller

Whenever you have finished with this notebook, be sure to terminate the controller. This is especially important if your business logic runs across multiple notebooks.

In [15]:
await agent_controller.terminate()